In [ ]:
import heapq
import math
from typing import List, Tuple

# from scipy.interpolate import interp1d
# from src.lib.cortex import cubic_spline_planner
import networkx as nx
import numpy as np
from copy import deepcopy
# from src.config import config

def dijkstra(G, start, target):
    d = {start: 0}
    parent = {start: None}
    pq = [(0, start)]
    visited = set()
    while pq:
        du, u = heapq.heappop(pq)
        if u in visited:
            continue
        if u == target:
            break
        visited.add(u)
        for v in G.adj[u]:
            if v not in d or d[v] > du + 1:
                d[v] = du + 1
                parent[v] = u
                heapq.heappush(pq, (d[v], v))

    fp = [target]
    tg = target
    ptype = [("lk" if len([i for i in G.neighbors(tg)]) < 2 else "int")]

    while tg != start:
        fp.insert(0, parent[tg])
        tg = parent[tg]
        ptype.insert(0, ("lk" if len([i for i in G.neighbors(tg)]) < 2 else "int"))
        # print([i for i in G.neighbors(tg)])

    ptyperet = ptype.copy()
    for i in range(len(ptype)):
        if ptype[i] == "int":
            try:
                ptyperet[i - 1] = "int"
            except Exception as e:
                print("pathplanning.py[46]",e)

            try:
                ptyperet[i + 1] = "int"
                i += 1
            except Exception as e:
                print("pathplanning.py[52]",e)
                

    edgeret = []

    for i in range(len(fp) - 1):
        dt = G.get_edge_data(fp[i], fp[i + 1])
        edgeret.append(dt["dotted"])

    edgeret.append(None)

    return fp, ptyperet, edgeret

def add_yaw(G):
    node_dict = deepcopy(dict(G.nodes(data=True)))
    for current_node in node_dict:
        for v in G.adj[current_node]:
            c_data = node_dict[current_node]
            v_data = node_dict[v]
            dx = v_data["x"] - c_data["x"]
            dy = v_data["y"] - c_data["y"]
            # print(c_data, v_data, "\n", dx, dy)
            yaw = math.atan2(dy, dx)
            if "yaw" in node_dict[current_node]:
                node_dict[current_node]["yaw"] += [yaw]
            else:
                node_dict[current_node].update({"yaw": [yaw]})
    return node_dict


class Navigator():
    
    def __init__(self,test=False):
        if test:
            self.graph = nx.read_graphml(
                "./src/lib/cortex/path_data/test_track.graphml"
            )
        else:
            self.graph = nx.read_graphml(
                "./src/lib/cortex/path_data/comp_track.graphml"
            )

        self.node_dict = add_yaw(self.graph)
        self.cur_index=0
        self.path=[]
        self.ptype=[]
        self.etype=[]
        
    def plan_course(self,config_json):
        raise NotImplementedError
    
    def replan_course(self,completed_list,config_json):
        raise NotImplementedError
        
    def get_course_ahead(self,x,y,yaw):
        raise NotImplementedError

    def get_current_node(self):
        raise NotImplementedError
        
    def get_nearest_node(self, x, y, yaw):
        dx = []
        dy = []
        for node in self.node_dict:
            dx.append(self.node_dict[node]["x"] - x)
            dy.append(self.node_dict[node]["y"] - y)

        d = np.hypot(dx, dy)
        idxs = np.argsort(d)
        for idx in idxs:
            try:
                dyaw = np.array(self.node_dict[str(idx)]["yaw"]) - yaw
            except KeyError as e:
                print(e)
                continue
            if (abs(dyaw) < 0.7).any():
                return idx  # , self.node_dict[str(idx)]
    
    
    def get_path_ahead(self,x,y,yaw):
        
        idx=self.get_nearest_node(self,x,y,yaw)
        return self._convert_nx_path2list(path_list[idx:]), _ptype[idx:],_edgret[idx:]
    
    def replan_path(self,end_idx):
        
        idx=self.get_nearest_node(self,x,y,yaw)
        path_list, _ptype, _edgret = dijkstra(self.graph, idx, end_idx)
        return self._convert_nx_path2list(path_list), _ptype,_edgret
        
            
    def get_path(self, start_idx: str, end_idx: str) -> Tuple[List[Tuple[int]], str]:
        
        path_list, _ptype, _edgret = dijkstra(self.graph, start_idx, end_idx)
        return self._convert_nx_path2list(path_list), _ptype,_edgret

    def _convert_nx_path2list(self, path_list) -> List[Tuple[int]]:
        coord_list = []
        for i in path_list:
            data = self.node_dict[i]
            coord_list.append([data["x"], data["y"]])
        return coord_list

    
    
        

In [ ]:
import heapq
import math
from typing import List, Tuple

# from scipy.interpolate import interp1d
from src.lib.cortex import cubic_spline_planner
import networkx as nx
import numpy as np
from copy import deepcopy
from src.config import config

def dijkstra(G, start, target):
    d = {start: 0}
    parent = {start: None}
    pq = [(0, start)]
    visited = set()
    while pq:
        du, u = heapq.heappop(pq)
        if u in visited:
            continue
        if u == target:
            break
        visited.add(u)
        for v in G.adj[u]:
            if v not in d or d[v] > du + 1:
                d[v] = du + 1
                parent[v] = u
                heapq.heappush(pq, (d[v], v))

    fp = [target]
    tg = target
    ptype = [("lk" if len([i for i in G.neighbors(tg)]) < 2 else "int")]

    while tg != start:
        fp.insert(0, parent[tg])
        tg = parent[tg]
        ptype.insert(0, ("lk" if len([i for i in G.neighbors(tg)]) < 2 else "int"))
        # print([i for i in G.neighbors(tg)])

    ptyperet = ptype.copy()
    for i in range(len(ptype)):
        if ptype[i] == "int":
            try:
                ptyperet[i - 1] = "int"
            except Exception as e:
                print("pathplanning.py[46]",e)

            try:
                ptyperet[i + 1] = "int"
                i += 1
            except Exception as e:
                print("pathplanning.py[52]",e)
                

    edgeret = []

    for i in range(len(fp) - 1):
        dt = G.get_edge_data(fp[i], fp[i + 1])
        edgeret.append(dt["dotted"])

    edgeret.append(None)

    return fp, ptyperet, edgeret


def add_yaw(G):
    node_dict = deepcopy(dict(G.nodes(data=True)))
    for current_node in node_dict:
        for v in G.adj[current_node]:
            c_data = node_dict[current_node]
            v_data = node_dict[v]
            dx = v_data["x"] - c_data["x"]
            dy = v_data["y"] - c_data["y"]
            # print(c_data, v_data, "\n", dx, dy)
            yaw = math.atan2(dy, dx)
            if "yaw" in node_dict[current_node]:
                node_dict[current_node]["yaw"] += [yaw]
            else:
                node_dict[current_node].update({"yaw": [yaw]})
    return node_dict

def give_perpendicular_park_pts(x,y,spot=1):
    # if spot==1:
    #     pt0x,pt0y=x+0.01,y+0.01
    #     pt1x,pt1y=x+0.5588,pt0y-0.5334
    #     pt2x,pt2y=pt1x+0.508,pt1y+0.3302
    #     pt3x,pt3y=pt2x-0.127,pt2y+0.66
    #     pt4x,pt4y=pt3x+0,pt3y+0.2032
    #     pt5x,pt5y=pt4x,pt4y+0.2032
    #     park_x_n,park_y_n=[pt0x,pt1x,pt2x,pt3x,pt4x,pt5x],[pt0x,pt1y,pt2y,pt3y,pt4y,pt5y]
    #     cx,cy,cyaw,rk,s=cubic_spline_planner.calc_spline_course(park_x_n,park_y_n,ds=0.15)
       
    #     return [i for i in zip(cx,cy)]
    #temporary:
    coord_list = [
        (2.1199999999999997, 2.1199999999999997),
        (2.2072555826316647, 1.9642686239127372),
        (2.2987501548820624, 1.8198978465696687),
        (2.398722706369926, 1.6982482667149883),
        (2.511412226713987, 1.6106804830928898),
        (2.6410577055329796, 1.5685550944475672),
        (2.7895393410766838, 1.581190302937868),
        (2.9415339509365492, 1.6430084931545883),
        (3.0740966676307324, 1.7418326938602926),
        (3.1642485373345357, 1.8654564196217684),
        (3.192618769936659, 2.002381602029436),
        (3.1697268696723078, 2.146978751551605),
        (3.1208636337525117, 2.2965185340365264),
        (3.171424620664199, 2.348292183875666),
        (3.1565564782777963, 2.499609817049711),
        (3.1594334941971126, 2.6494483048050387),
        (3.1507302101554803, 2.753755087166564),
    ]

    return coord_list

class PathPlanning:
    def __init__(self, test: bool = False) -> None:
        if test:
            self.graph = nx.read_graphml(
                "./src/lib/cortex/path_data/test_track.graphml"
            )
        else:
            self.graph = nx.read_graphml(
                "./src/lib/cortex/path_data/comp_track.graphml"
            )

        self.node_dict = add_yaw(self.graph)

    def get_path(self, start_idx: str, end_idx: str) -> Tuple[List[Tuple[int]], str]:
        
        path_list, _ptype, _edgret = dijkstra(self.graph, start_idx, end_idx)

        return self._convert_nx_path2list(path_list), _ptype,_edgret

    def get_nearest_node(self, x, y, yaw):
        dx = []
        dy = []
        for node in self.node_dict:
            dx.append(self.node_dict[node]["x"] - x)
            dy.append(self.node_dict[node]["y"] - y)

        d = np.hypot(dx, dy)
        idxs = np.argsort(d)
        for idx in idxs:
            try:
                dyaw = np.array(self.node_dict[str(idx)]["yaw"]) - yaw
            except KeyError as e:
                print(e)
                continue
            if (abs(dyaw) < 0.7).any():
                return idx  # , self.node_dict[str(idx)]

    def _convert_nx_path2list(self, path_list) -> List[Tuple[int]]:
        coord_list = []
        for i in path_list:
            data = self.node_dict[i]
            coord_list.append([data["x"], data["y"]])
        return coord_list
